In [13]:
import creds
import requests
from bs4 import BeautifulSoup
import pandas as pd

date_out = input('Enter a date formatted as YYYY-MM-DD: ') ## Variable global para la fecha que quieras exportar

def GetResponseUrl(): #### esta funcion te da el url de response

    date_components =  date_out.split("-")

    year, month, day = [int(item) for item in date_components]
    
    f_url = 'https://app.damageid.com/case/list?searchUnit=&inoutKey=dateOut&inoutValue='

    date = f'{month}%2F{day}%2F{year}'

    c_url = '&statusFilter=-1&templateFilter=7&searchCaseNumber=&searchUser=&searchLicensePlate=&searchVin='

    response_url = f_url + date + c_url

    return response_url


request_url = ('https://app.damageid.com/authenticate')

response_url = (GetResponseUrl())

payload = { #### tienes que tener el archivo creds.py en el mismo folder que este notebook
    
    'username':creds.username,
    'password':creds.password    
    }

case_list = []
status_list = []
van_list = []
username = []

print("Scraping DamageID...")
with requests.session() as s: 
        s.post(request_url, data = payload)
        r = s.get(response_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        cases = soup.find_all('td', {'class':'damage-td-first'})
        statuses = soup.find_all('td', {'class':'damage-td-last'})
        table = soup.find('table', {"class":"table table-hover"})

        for case in cases:
            case_list.append(case.text.replace("\t","").replace("\n","").strip())
        
        
        for status in statuses:
                status_list.append(status.text.strip())
        
        for driver in table.find_all("tbody"):
            rows = driver.find_all("tr")
            for row in rows:
                values = row.find("td")
                    
                values1 = values.find_next("td")
                van_list.append(values1.text.strip())
                   
                values2 = values1.find_next("td")
                values2 = values2.text.split(" ")[0].strip()
                username.append(values2)

        
        df = pd.DataFrame({"Cases ID":case_list, "Van Unit":van_list, "Username":username, "Status":status_list})
        
        df["Date_Out"] = date_out
print("Complete")

a = input("export? y/n")

if a == "y":
    file = "DamageID_" + date_out + ".csv"
    df.to_csv(file)
    print(f"File {file} saved")
else:
    print(df)

Enter a date formatted as YYYY-MM-DD:  2022-12-12


Scraping DamageID...
Complete


export? y/n y


File DamageID_2022-12-12.csv saved
